In [1]:
import numpy as np
from scipy import interpolate
import math
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import matplotlib
import matplotlib.cm as cm
import matplotlib.patheffects as path_effects
matplotlib.rc('font', family='sans-serif', size=12)
matplotlib.rcParams['xtick.direction'] = 'in'
matplotlib.rcParams['ytick.direction'] = 'in'
matplotlib.rcParams['xtick.top'] = True
matplotlib.rcParams['ytick.right'] = True
matplotlib.rcParams['xtick.minor.visible'] = True
matplotlib.rcParams['ytick.minor.visible'] = True
matplotlib.rcParams['lines.dash_capstyle'] = 'round'

matplotlib.rcParams['lines.solid_capstyle'] = 'round'

cycle=plt.rcParams['axes.prop_cycle'].by_key()['color']

import matplotlib.colors as colors
from matplotlib import gridspec

from scipy.interpolate import interp1d
import scipy.integrate as integrate
import scipy.optimize 

In [2]:
# define a function that returns value of the cooling curve at any given value of temperature and pressure (or, alternatively, vz)
def edot_cool_double_equilibrium(T, vz):
    pressure = T * mdot / vz
    alpha_heat          = (np.log10(T_cold/T_peak)/np.log10(density_contrast) * (beta_lo - beta_hi)) - beta_hi
    heating_coefficient = (T_cold/T_peak)**((beta_hi-beta_lo)*(1.0 + (np.log10(T_cold/T_peak)/np.log10(density_contrast))))
    Edot_cool = 1.5 * P_hot/t_cool_min * (T/T_peak)**np.where(T<T_peak, -beta_lo, -beta_hi) * (pressure/P_hot)**2 #* (T/T_peak)**np.where(T<T_peak, -beta_lo, -beta_hi)
    Edot_heat = 1.5 * P_hot/t_cool_min * heating_coefficient * (pressure/P_hot)**2
    Edot_heat *= np.where(T<(1+epsilon_T)*T_hot, (T/T_peak)**alpha_heat, ((1+epsilon_T)*T_hot/T_peak)**alpha_heat * (T/((1+epsilon_T)*T_hot))**(-beta_hi-0.5))
    return (Edot_cool - Edot_heat)


# define functions that return values of vx and its 1st & 2nd derivatives 
# remember that we are taking the vx profile as a cosine
def vx_cos(z):
    if (z > h):
        return 0
    else:
        return (vrel/2)*(np.cos(np.pi*z/h)+1)

vx_cos = np.vectorize(vx_cos)    
    
def dvx_dz_cos(z):
    if (z > h):
        return 0
    else:
        return -(vrel/2) * np.pi/h * np.sin(np.pi*z/h)

dvx_dz_cos = np.vectorize(dvx_dz_cos)
    
def d2vx_dz2_cos(z):
    if (z > h):
        return 0
    else:
        return -(vrel/2) * (np.pi/h)**2 * np.cos(np.pi*z/h)

d2vx_dz2_cos = np.vectorize(d2vx_dz2_cos)


# define an integrator that will later be taken as an input to scipy's solve_ivp function.
# we are solving 4 coupled differential equations (1st & 2nd derivatives of T and vz), so we need expressions of those as outputs of the integrator function
def integrator(z, w, mdot): 
    dT_dz, T, dvz_dz, vz = w
    if (z <= 0):
        dvx_dz = 0
        d2vx_dz2 =0 
    elif (z > h):
        dvx_dz = 0
        d2vx_dz2 =0 
    else:
        dvx_dz = -(vrel/2) * np.pi/h * np.sin(np.pi*z/h)
        d2vx_dz2 = -(vrel/2) * (np.pi/h)**2 * np.cos(np.pi*z/h)

    if (T <= T_peak):
        pressure = T * mdot / vz
        edot_cool = 1.5 * P_hot/t_cool_min * (pressure/P_hot)**2 * ((T/T_peak)**-beta_lo - heating_coefficient*(T/T_peak)**alpha_heat)
    elif (T > T_peak):
        pressure = T * mdot / vz
        edot_cool = 1.5 * P_hot/t_cool_min * (pressure/P_hot)**2 * ((T/T_peak)**-beta_hi - heating_coefficient*(T/T_peak)**alpha_heat)
    
    # the derivation of these equations can be found in the methods section of the paper
    kappa        = (mdot/vz) * (f_nu * h**2 * np.abs(dvx_dz) + kappa_0 / (mdot/vz))
    mu        = Prandtl * (mdot/vz) * (f_nu * h**2 * np.abs(dvx_dz) + kappa_0 / (mdot/vz))
    dkappa_dz       = f_nu * h**2 * ((mdot/vz) * d2vx_dz2 - (mdot/vz**2) * np.abs(dvx_dz) * dvz_dz)
    dmu_dz       = Prandtl * (f_nu * h**2 * ((mdot/vz) * d2vx_dz2 - (mdot/vz**2) * np.abs(dvx_dz) * dvz_dz))
    d2T_dz2     = edot_cool_double_equilibrium(T, vz)/kappa + mdot*vz/kappa * (T/vz**2 * dvz_dz + dT_dz/(gamma-1)/vz) - dT_dz * dkappa_dz/kappa - mu/kappa * (dvx_dz**2 + (4/3.)*dvz_dz**2)
    d2vz_dz2    = (3./4.) * (mdot/mu) * (dT_dz/vz + dvz_dz*(1-T/vz**2)) - dvz_dz * dmu_dz/mu
    return np.array([d2T_dz2, dT_dz, d2vz_dz2, dvz_dz])


# we define two termination events that will later be taken as inputs to the solve_ivp function.
# we want to terminate the integration if temperature drops below the cold phase or exceeds the hot phase.
def dip(z,w,mdot): # terminate when T drops below T_cold
    T = w[1]
    return T/T_cold - 0.999

dip.terminal = True

def bump(z,w,mdot): # terminate when T exceeds T_hot
    T = w[1]
    return T/T_hot - 1.001

bump.terminal = True


# use the integrator and the termination events to integrate a solution for a guess of mdot
# this function then returns the final T gradient of the solution, which is a crucial criterion that informs our bisection process
def find_final_gradient(mdot_over_mdot_crit):
    mdot                = mdot_crit * mdot_over_mdot_crit
    T_initial           = T_hot
    vz_initial          = mdot/rho_hot
    dT_dz_initial       = -1e-6
    dvz_dz_initial      = 1e-6
    initial_conditions  = [dT_dz_initial, T_initial, dvz_dz_initial, vz_initial]
    stop_distance       = 10**4
    sol = solve_ivp(integrator, [0, stop_distance], initial_conditions, 
        dense_output=True, 
        events=[dip, bump],
        rtol=3e-14, atol=[1e-9,1e-11,1e-9,1e-11],
        args=[mdot])
    return sol.y[0][-1]


# similar to the find_final_gradient function, but this function returns the solution object from the solve_ivp function, which is useful for plotting
def calculate_solution(mdot_over_mdot_crit):
    mdot                = mdot_crit * mdot_over_mdot_crit
    T_initial           = T_hot
    vz_initial          = mdot/rho_hot
    dT_dz_initial       = -1e-6
    dvz_dz_initial      = 1e-6
    initial_conditions  = [dT_dz_initial, T_initial, dvz_dz_initial, vz_initial]
    stop_distance       = 10**4
    sol = solve_ivp(integrator, [0, stop_distance], initial_conditions, 
        dense_output=True, 
        events=[dip, bump],
        rtol=3e-14, atol=[1e-9,1e-11,1e-9,1e-11],
        args=[mdot])
    return sol





In [3]:

# define a function that makes a 4-panel plot for a solution
# the plot includes the T, P, vz, and vx profile, the phase distributions, and the terms decomposition in position and temperature space

def plot_solution(sol,mdot_over_mdot_crit,name=None):
    mdot           = mdot_crit * mdot_over_mdot_crit
    z           = sol.t
    dT_dz       = sol.y[0]
    T           = sol.y[1]
    dvz_dz      = sol.y[2]
    vz          = sol.y[3]
    P           = T*(mdot/vz)
    rho         = (mdot/vz)
    vx          = vx_cos(z)
    dvx_dz      = dvx_dz_cos(z)
    d2vx_dz2    = d2vx_dz2_cos(z)
    
    dEdot_dlogT = (T/-dT_dz)*edot_cool_double_equilibrium(T, vz)
    dM_dlogT    = (T/-dT_dz)*rho

    kappa        = (mdot/vz) * (f_nu * h**2 * np.abs(dvx_dz) + kappa_0 / (mdot/vz))
    mu        = Prandtl * (mdot/vz) * (f_nu * h**2 * np.abs(dvx_dz) + kappa_0 / (mdot/vz))
    dkappa_dz       = f_nu * h**2 * ((mdot/vz) * d2vx_dz2 - (mdot/vz**2) * np.abs(dvx_dz) * dvz_dz)
    dmu_dz       = Prandtl * (f_nu * h**2 * ((mdot/vz) * d2vx_dz2 - (mdot/vz**2) * np.abs(dvx_dz) * dvz_dz))
    d2T_dz2     = edot_cool_double_equilibrium(T, vz)/kappa + mdot*vz/kappa * (T/vz**2 * dvz_dz + dT_dz/(gamma-1)/vz) - dT_dz * dkappa_dz/kappa - mu/kappa * (dvx_dz**2 + (4/3.)*dvz_dz**2)
    d2vz_dz2    = (3./4.) * (mdot/mu) * (dT_dz/vz + dvz_dz*(1-T/vz**2)) - dvz_dz * dmu_dz/mu
    
    dHvisc_dlogT = (T/-dT_dz)*mu*(dvx_dz**2 + (4/3.)*dvz_dz**2)
    dP_dz = (mdot/vz)*dT_dz - (mdot*T/vz**2)*dvz_dz
    dWork_dlogT = (T/-dT_dz)*vz*dP_dz
    
    
    adv_enthalpy = (gamma/(gamma-1))*mdot*dT_dz
    
    dkappa_dz_dT_dz = dkappa_dz * dT_dz
    conduction = dkappa_dz_dT_dz + kappa*d2T_dz2
    
    cooling = -edot_cool_double_equilibrium(T, vz)
    
    x_visc_heating = mu*(dvx_dz)**2
    z_visc_heating = mu*(4.0/3)*dvz_dz**2
    
    work = mdot*(dT_dz - dvz_dz*T/vz)
    
    
    fig, ((ax1,ax3),(ax2,ax4)) = plt.subplots(2,2)

    plt.rcParams.update({
        "text.usetex": True,
        "font.family": "serif",
        "font.serif": ["Computer Modern Roman"],
    })


    ax1.plot(z,  T, color='#2A3132', label=r'$T  $')
    ax1.plot(z,  vz, color='#FF9C44', label=r'$v_z $')
    ax1.plot(z,  vx, color='#EF476F', label=r'$v_x $')
    ax1.plot(z,  P, color='#88B04B', label=r'$P  $')

    ax1.set_ylabel('Profiles', fontsize=8)

    ax1.annotate(r'$\frac{T}{T_{\rm hot}}$', xy=(0.23, 0.7), color='#2A3132',fontsize=8,
                xytext=(0.29, 0.5), textcoords='axes fraction',
                horizontalalignment='right', verticalalignment='top')

    ax1.annotate(r'$\frac{P}{P_{\rm hot}}$', xy=(0.5, 0.94), color='#88B04B',fontsize=8,
                xytext=(0.5, 0.94), textcoords='axes fraction',
                horizontalalignment='right', verticalalignment='top')

    ax1.annotate(r'$\frac{v_z}{c_{\rm s,hot}}$', xy=(0.1, 0.26), color='#FF9C44',fontsize=8,
                xytext=(0.075, 0.22), textcoords='axes fraction',
                horizontalalignment='center', verticalalignment='center')

    ax1.annotate(r'$\frac{v_x}{c_{\rm s,hot}}$', xy=(0.8, 0.57), color='#EF476F',fontsize=8,
                xytext=(0.8, 0.2), textcoords='axes fraction',
                horizontalalignment='center', verticalalignment='center')




    ax2.semilogy(z,  adv_enthalpy,   color='#2A3132',              label='adv. enth.')
    ax2.semilogy(z,  conduction,            color='#702A70', label='conduction')
    ax2.semilogy(z,  cooling,  color='#118AB2', label='cooling')
    ax2.semilogy(z,  x_visc_heating,                      color='#EF476F', label='x visc. heat.')
    ax2.semilogy(z,  z_visc_heating,               color='#FF9C44', label='z visc. heat.')
    ax2.semilogy(z,  work,               color='#88B04B', label='work')

    ax2.semilogy(z,  -adv_enthalpy,   color='#2A3132', ls='--')
    ax2.semilogy(z,  -conduction,         color='#702A70', ls='--')
    ax2.semilogy(z,  -cooling,   color='#118AB2', ls='--')
    ax2.semilogy(z,  -x_visc_heating,                     color='#EF476F', ls='--')
    ax2.semilogy(z,  -z_visc_heating,              color='#FF9C44', ls='--')
    ax2.semilogy(z,  -work,              color='#88B04B', ls='--')

    ax2.set_ylabel(r'$\left. \dot \varepsilon \right/ \dot \varepsilon_0$')


    ax3.loglog(T,  dEdot_dlogT, color='#118AB2', label=r'$\frac{d \dot{E}}{d \log T}$')
    ax3.loglog(T,  dM_dlogT, color='#743023', label=r'$\frac{d M}{d \log T}$')
    i_lo = np.argmin(np.abs(T-1.2*T_cold))
    i_hi = np.argmin(np.abs(T-0.8*T_hot))
    maximum = np.max([np.max(dEdot_dlogT[i_hi:i_lo]),np.max(dM_dlogT[i_hi:i_lo])])
    minimum = np.min([np.min(np.abs(dEdot_dlogT)[i_hi:i_lo]),np.min(np.abs(dM_dlogT)[i_hi:i_lo])])
    ax3.set_ylim((minimum,maximum))

    ax3.set_ylabel('Phase Distributions', fontsize=8)

    ax3.annotate(r'$\frac{1}{\dot{E}_0}\frac{d \dot E_{\rm cool} }{d logT}$', xy=(0.3, 0.25), color='#118AB2',fontsize=8,
                xytext=(0.3, 0.2), textcoords='axes fraction',
                horizontalalignment='center', verticalalignment='center')

    ax3.annotate(r'$\frac{1}{M_0} \frac{d M}{d logT}$', xy=(0.5, 0.45), color='#743023',fontsize=8,
                xytext=(0.5, 0.45), textcoords='axes fraction',
                horizontalalignment='center', verticalalignment='center')


    ax4.loglog(T,  adv_enthalpy,   color='#2A3132', label='adv. enth.')
    ax4.loglog(T,  conduction,            color='#702A70', label='conduction')
    ax4.loglog(T,  cooling,  color='#118AB2', label='cooling')
    ax4.loglog(T,  x_visc_heating,                      color='#EF476F', label='x visc. heat.')
    ax4.loglog(T,  z_visc_heating,               color='#FF9C44', label='z visc. heat.')
    ax4.loglog(T,  work,               color='#88B04B', label='work')

    ax4.loglog(T,  -adv_enthalpy,   color='#2A3132', ls='--')
    ax4.loglog(T,  -conduction,         color='#702A70', ls='--')
    ax4.loglog(T,  -cooling,   color='#118AB2', ls='--')
    ax4.loglog(T,  -x_visc_heating,                     color='#EF476F', ls='--')
    ax4.loglog(T,  -z_visc_heating,              color='#FF9C44', ls='--')
    ax4.loglog(T,  -work,              color='#88B04B', ls='--')

    ax4.set_ylabel(r'$\left. \dot \varepsilon \right/ \dot \varepsilon_0$')

    ax2.set_ylim((2e-4,3e1))
    ax4.set_ylim((2e-4,3e1))



    ax2.set_xlabel(r'$\left. z \right/ h$', fontsize=8)
    ax4.set_xlabel(r'$\left. T \right/ T_{\rm hot}$', fontsize=8)


    ax2.legend(loc='upper right', ncol = 2, fontsize=6, handlelength=1.3, labelspacing=0.25, columnspacing=0.7)
    ax4.legend(loc='lower center', ncol = 3, fontsize=6, handlelength=1.3, labelspacing=0.25, columnspacing=0.7)

    plt.xticks(fontsize=8)
    plt.yticks(fontsize=8)

    ax1.tick_params(axis='both', labelsize=8)
    ax2.tick_params(axis='both', labelsize=8)
    ax3.tick_params(axis='both', labelsize=8)
    ax4.tick_params(axis='both', labelsize=8)

    fig.set_size_inches(7.1, 7.1*(2/3))

    plt.subplots_adjust(hspace=0.02, wspace=0.25)




    
    if name:
        plt.savefig(name+'.pdf',dpi=200, bbox_inches='tight')
        plt.show()
    else:
        plt.show()
    plt.close('all')
    return


# ----------------------

In [ ]:
# set the relative shear velocity between the phases here, note that in our setup cs,hot=sqrt(gamma)=sqrt(5/3)
vrel                = 0.75

# set the width of the mixing layer here, the fiducial value is 1 
h                   = 1

# set the value of f_nu here, suggested value is between 10**(-2.5) and 10**(-1.5)
f_nu                = 0.01

# set the value of kappa_0 here. kappa_0 is the constant term in the expression of kappa that prevents singularities in the solution
# the fiducial value is 10**(-3)
kappa_0             = 1e-3

# set the value of the Prandtl number here, suggested value is between 0.1 and 1
Prandtl             = 0.3

# set the value of t_cool_min here, suggested value is between 10**(-2) and 10**(-1)
t_cool_min          = 10**(-1)

In [ ]:
# define some constants that are used in the integration
# note that we need initial T gradient to be small and non-zero to give the solution a "nudge" away from the equilibrium at the hot phase temperature

dT_dz_initial       = -1e-6
dvz_dz_initial      = 0


beta_lo             = -2
beta_hi             = 1
density_contrast    = 100.
T_peak_over_T_cold  = density_contrast**(1/3.)

gamma               = 5.0/3.0 
P_hot               = 1.0
T_cold              = 1.0/density_contrast
T_hot               = 1.0
T_peak              = T_peak_over_T_cold * T_cold
epsilon_T           = 0.005
rho_hot             = P_hot / T_hot
mdot_crit           = rho_hot * np.sqrt(T_hot)
alpha_heat          = (np.log10(T_cold/T_peak)/np.log10(density_contrast) * (beta_lo - beta_hi)) - beta_hi
heating_coefficient = (T_cold/T_peak)**((beta_hi-beta_lo)*(1.0 + (np.log10(T_cold/T_peak)/np.log10(density_contrast))))

In [ ]:
# we perform our bisection to find the eigenvalue of the mass flux mdot here
# our first guess for mdot is the maximum possible value mdot_crit, which corresponds to a vz_initial that is equal to c_s,hot
mdot_over_mdot_crit = 1.0
factor = 0.9
final_gradient = find_final_gradient(mdot_over_mdot_crit)
# we record the sign of the final T gradient of this solution, decrase our guess of mdot by 10%, and repeat this process
positive = 1
# we repeat the above procedure until the final T gradient changes sign
while positive > 0:
    mdot_over_mdot_crit *= factor
    next_final_gradient = find_final_gradient(mdot_over_mdot_crit)
    print(next_final_gradient)
    positive = next_final_gradient*final_gradient
    final_gradient = next_final_gradient
    
# now we know that the eigenvalue of mdot must be sandwiched between the two most recent guesses of mdot
# given this information, we can use scipy's root finder optimize.root_scalar to find the eigenvalue of mdot (mdot that corresponds to final T gradient=0)
# note that we set rtol and xtol to be very small here so that we can resolve the eigenvalue as accurately as possible
target = scipy.optimize.root_scalar(find_final_gradient, bracket = [mdot_over_mdot_crit,mdot_over_mdot_crit/factor], xtol=1e-14, rtol=1e-14)
sol = calculate_solution(target.root)

# finally we plot the solution we obtained using the pre-defined plotting function
plot_solution(sol,target.root)

